In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
## 설정
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS     = 40
BATCH_SIZE = 64

In [3]:
## 데이터
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(), # 과적합 방지를 위해 RandomCrop과 RandomHorizontalFlip을 추가했습니다.
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])), # 각 채널별 평균을 뺀 뒤 표준편차로 나누어 정규화를 진행합니다.((R채널 평균, G채널 평균, B채널 평균), (R채널 표준편차, G채널 표준편차, B채널 표준편차))
    batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


In [4]:
## 클래스
class BasicBlock(nn.Module): # Residual 블록을 BasicBlock이라는 새로운 파이토치 모듈로 정의해서 사용합니다.
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        # ResNet의 두번째 블록 부터는 in_planes()를 받아 self.bn2 계층과 출력크기가 같은 planes를 더해주는 self.shortcut모듈을 정의합니다
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16 # self.in_planes 변수는 _make_layer 함수가 층을 만들 때 채널 출력값을 기록하는 데 쓰입니다.
        # layer1이 입력받는 채널의 개수가 16개이므로 16으로 초기화해줍니다.

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1) 
        #self._make_layer()는 nn.Sequential의 도구로 여러 BasicBlock 모듈을 하나로 묶어주는 역할을 합니다.
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        # self.layer1~3은 컨볼루션 계층과 마찬가지로 모듈(nn.Module)로 취급하면 됩니다.
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    # _make_layer 함수는 self.in_planes 채널 개수로부터 직접 입력받은 인수인 planes 채널 개수만큼을 출력하는 BasicBlock을 생성합니다.
    # layer1 : 16채널에서 16채널을 보내는 BasicBlock 2개
    # layer2 : 16채널을 받아 32채널을 출력하는 BasicBlock 1개와 32채널에서 32채널을 내보내는 BasicBlock 1개
    # layer3 : 32채널을 받아 64채널을 출력하는 BasicBlock 1개와 64채널에서 64채널을 출력하는 BasicBlock 1개

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    # ResNet 모델은 위와같이 컨볼루션, 배치정규화, 활성화 함수를 통과하고 사전에 정의해둔 BasicBlock 층을 가지고 있는
    # layer1, layer2, layer3를 통과하게 됩니다. 각 layer는 2개의 Residual 블록을 갖고 있고 
    # 이렇게 나온 값에 평균 풀링을 하고 마지막 계층을 거쳐 분류결과를 출력합니다.

In [8]:
## 학습 & 평가
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005) 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# scheduler는 이폭마다 호출되며 step_size를 50으로 지정해 50번 호출될 때 학습률에 0.1(gamma)만큼 곱합니다.
# 즉 0.1로 시작한 학습률은 50이폭 이후에 0.1 x 0.1 = 0.01로 낮아집니다.

In [10]:
#@ 훈련
for epoch in range(1, EPOCHS + 1):
    scheduler.step() #코드는 위와 대부분 동일하나 scheduler.step()으로 학습률을 조금 낮춰주는 단계가 추가되었습니다.
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.441961
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.627731
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.659382
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.624555
[1] Test Loss: 1.4757, Accuracy: 48.66%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.395203
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.033806
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.085899
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.945848
[2] Test Loss: 1.0821, Accuracy: 62.85%
Train Epoch: 3 [0/50000 (0%)]	Loss: 0.942703
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.087417
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.965589
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.856370
[3] Test Loss: 1.0313, Accuracy: 65.03%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.677473
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.708026
Train Epoch: 4 [25600/50000 (51%)]	Loss: 1.013643
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.771932
[4] Test Loss: 0.8618, Accuracy: 70.93%
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.026476
Train Epoch: 5 

Train Epoch: 35 [25600/50000 (51%)]	Loss: 0.635907
Train Epoch: 35 [38400/50000 (77%)]	Loss: 0.551443
[35] Test Loss: 0.8543, Accuracy: 72.41%
Train Epoch: 36 [0/50000 (0%)]	Loss: 0.410629
Train Epoch: 36 [12800/50000 (26%)]	Loss: 0.638334
Train Epoch: 36 [25600/50000 (51%)]	Loss: 0.615096
Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.591607
[36] Test Loss: 0.7961, Accuracy: 73.96%
Train Epoch: 37 [0/50000 (0%)]	Loss: 0.819165
Train Epoch: 37 [12800/50000 (26%)]	Loss: 0.679754
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.982552
Train Epoch: 37 [38400/50000 (77%)]	Loss: 0.896366
[37] Test Loss: 1.2577, Accuracy: 64.46%
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.498359
Train Epoch: 38 [12800/50000 (26%)]	Loss: 0.547946
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.754767
Train Epoch: 38 [38400/50000 (77%)]	Loss: 0.519863
[38] Test Loss: 1.0575, Accuracy: 67.69%
Train Epoch: 39 [0/50000 (0%)]	Loss: 0.822788
Train Epoch: 39 [12800/50000 (26%)]	Loss: 0.533716
Train Epoch: 39 [25600/50000 (51%)]	Loss